## Creating DF for IDs


In [2]:
import requests
from pandas.io.json import json_normalize
from pandas import DataFrame
import pandas as pd 
import time
from datetime import datetime

In [3]:
access_token = ''
req = requests.get('https://api.crowdtangle.com/lists?token='+access_token)

In [5]:
df_id = pd.DataFrame(columns=['type','id','title'])
types = ['SAVED_SEARCH']
ids = req.json()['result']['lists']
num = 0
for i in range(len(ids)):
  if(ids[i]['type'] in types):
    df_id.loc[num] = ''
    df_id['type'][num] = ids[i]['type']
    df_id['id'][num] = ids[i]['id']
    df_id['title'][num] = ids[i]['title']
    num +=1

In [18]:
icount = '100'
access_token = ''
temp = requests.get('https://api.crowdtangle.com/posts?token='+access_token+'&listIds=1416869&count=100&startDate=2020-10-05T00:00:00&endDate=2020-10-05T23:59:59&offset=0&sortBy=oldest')

In [20]:
%%time
import re
list_ids = '1416869'
start_date = '2020-10-05 00:00:00'
start_date = re.sub(' ','T',start_date)
end_date = '2020-10-05 23:59:59'
end_date = re.sub(' ','T',end_date)
icount = '100'
access_token = ''

df_post_data = pd.DataFrame(columns=['Tool','date','handle','name','platform','subscriberCount','post_type','postUrl','actual_commentCount','actual_favoriteCount','expected_commentCount','expected_favoriteCount'])
num = 0
count = 0
pagination_flag = True
while (pagination_flag):
  print('Search Indexed old start:',start_date)
  req_list = requests.get('https://api.crowdtangle.com/posts?token='+access_token+'&listIds='+list_ids+'&count='+icount+'&startDate='+start_date+'&endDate='+end_date+'&sortBy=oldest')
  #new start date as the last value in the list of post and converting it to a format that API accepts
  start_date = req_list.json()['result']['posts'][-1]['date']
  start_date = re.sub(' ','T',start_date)
  print('Search Indexed new start:',start_date)
  print('Records retrived:',len(req_list.json()['result']['posts']))
  data = req_list.json()['result']['posts']
  for i in range(len(data)):
    df_post_data.loc[num]=''
    df_post_data['Tool'][num] = 'CrowdTangle'
    df_post_data['date'][num] = data[i]['date']
    df_post_data['handle'][num] = data[i]['account']['handle']
    df_post_data['name'][num] = data[i]['account']['name']
    df_post_data['subscriberCount'][num] = data[i]['account']['subscriberCount']
    df_post_data['platform'][num] = data[i]['account']['platform']
    if ('postUrl' in data[i].keys()): #checking if post URL is there or not
      df_post_data['postUrl'][num] = data[i]['postUrl']
    df_post_data['post_type'][num] = data[i]['type']
    df_post_data['actual_commentCount'][num] = data[i]['statistics']['actual']['commentCount']
    df_post_data['actual_favoriteCount'][num] = data[i]['statistics']['actual']['favoriteCount']
    df_post_data['expected_favoriteCount'][num] = data[i]['statistics']['expected']['favoriteCount']
    df_post_data['expected_commentCount'][num] = data[i]['statistics']['expected']['commentCount']

    num +=1
  count+=1
  if(count%5 == 0):
    time.sleep(61)
  if(len(req_list.json()['result']['pagination']) == 0):
    pagination_flag = False
  # req_list = requests.get('https://api.crowdtangle.com/posts?token='+access_token+'&listIds='+list_ids+'&count='+icount+'&startDate='+start_date_new+'&endDate='+end_date+'&sortBy=oldest')
  print(count)


Search Indexed old start: 2020-10-05T00:00:00
Search Indexed new start: 2020-10-05T23:59:13
Records retrived: 68
1
CPU times: user 443 ms, sys: 2.9 ms, total: 445 ms
Wall time: 1.37 s


In [21]:
print(df_post_data.shape)
df_post_data = df_post_data.drop_duplicates()
df_post_data = df_post_data.reset_index()
print(df_post_data.shape)

(68, 12)
(68, 13)


In [17]:
print(df_post_data.shape)
df_post_data = df_post_data.drop_duplicates()
df_post_data = df_post_data.reset_index()
print(df_post_data.shape)

(1, 13)
(1, 14)


In [23]:
df_post_data.to_csv('ct_5oct.csv')